# コマ分割作業用notebook
ここで試したものをスクリプト化したものが `split_frame.py`  
このときはまだデータセットの構成が定まっていなかった


# TODO

- [x] newgame以外でも動かせるように汎用化する
- [x] notebookからスクリプトに直す

NEWGAMEからコマを抽出する  
下記ブログを参考に  
[4コマ漫画のコマを切り抜く](http://atarabi.hateblo.jp/entry/2015/12/13/232415)

1. scansnapで裁断後のコミックスをスキャン
2. pdfimagesコマンドでpdfの全ページをjpeg化
3. openCVとpythonで4コマを判定・抽出

In [1]:
# scansnapは2値化されないよう，グレースケールになるように指定しておく
# 1冊取り込んだpdfに対してpdfimagesしてページ単位に分割する
! pdfimages -j data/NEWGAME05_CONTENT.pdf data/newgame5/5

# ImageMagickでもできるが結局ImageMagickはOS標準ではないし、IMに加えてgsが必要になる
# なんでpdfimagesを選んだか忘れたが、 MacOSなら`brew install poppler`で入る

In [2]:
import os
import numpy as np
from functools import cmp_to_key
import matplotlib.pyplot as plt
import cv2

In [3]:
def apply_adaptive_threshold(image, radius=15, C=5):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray, (15, 15), 0)
    #return cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    return cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 2 * radius + 1, C)

In [15]:
def find_external_contours(thresh):
    #contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    _, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    external_num = hierarchy.shape[1] if hierarchy is not None else 0
    return contours[0:external_num]
    
    contours = [c for c,h in zip(contours, hierarchy[0]) if h[3] > -1]
    return [cnt for cnt in contours if cv2.contourArea(cnt) > 3000]

In [5]:
def extract_rects_from_controus(contours, min_perimeter, max_perimeter):
    frames = []
    #for contour in contours:
    #    x,y,w,h = cv2.boundingRect(contour)
    #    frames.append(((x+w/2, y+h/2), (w+2, h+2), 0))
    #return frames
        
    for contour in contours:        
        frame = cv2.minAreaRect(contour)
        center, size, angle = frame
        # 縦・横が逆になっている場合、90度回転させる
        if angle < -45:
            size = tuple(reversed(size))
            angle = angle + 90
        w, h = size
        perimeter = 2 * (w + h)
        if min_perimeter < perimeter < max_perimeter and abs(angle) < 3.0 and 0.1 <= min(w, h) / max(w, h) <= 1.0:
            frames.append((center, (w + 2, h + 2), angle))  # パディングを加える
    return frames

In [6]:
def cmp_frame(tolerance):
    def _cmp(lhs, rhs):
        return (lhs > rhs) - (lhs < rhs)

    def _cmp_frame(lhs, rhs):
        if lhs[0] == rhs[0]:
            return 0
        x1, y1 = lhs[0]
        x2, y2 = rhs[0]
        if abs(x1 - x2) < tolerance:
            return _cmp(y1, y2)
        else:
            return _cmp(x2, x1)

    return _cmp_frame

In [19]:
def cut_frame(image, rect):
    center, size, angle = rect
    size = int(np.round(size[0])), int(np.round(size[1]))
    box = cv2.boxPoints(rect)
    M = cv2.getAffineTransform(np.float32(box[1:4]),  np.float32([[0, 0], [size[0], 0], [size[0], size[1]]]))
    return cv2.warpAffine(image, M, size)

In [8]:
def cut_frames(image):
    height, width, ch = image.shape 
    
    # 二値化
    thresh = apply_adaptive_threshold(image)

    ### add
    #lines = cv2.HoughLinesP(image = thresh, rho=0.1, theta = np.pi/180, 
    #                                            threshold = 100, minLineLength = 255, maxLineGap = 1)
    #num_pix_threshold = 500 # minimum number of pixels that must be on a line
    #lines = cv2.HoughLinesP(thresh, 1, np.pi/180, num_pix_threshold, minLineLength=300, maxLineGap=3)

    #line_image = np.zeros((height, width), dtype=np.uint8)
    #line_image.fill(255)
    #contours = []
    #if not lines is None:
    #    for x1,y1,x2,y2 in lines[0]:
    #        cv2.line(line_image, (x1,y1), (x2,y2), (0,0,0), 3)
    #    contours = find_external_contours(line_image)
    ### add
    
    # 一番外側の輪郭wだけを抽出
    contours = find_external_contours(thresh)

    # 抽出した輪郭からコマの四角形だけを取り出す
    min_perimeter, max_perimeter = (width + height) * 0.25,  (width + height) * 1.5
    rects = extract_rects_from_controus(contours, min_perimeter, max_perimeter)
    #f1 = lambda x: 150000 < x[1][0] * x[1][1] < 160000
    #f2 = lambda x: 600000 < x[1][0] * x[1][1] < 640000    
    #rects = filter(lambda x: f1(x) or f2(x), rects)

    # 抽出した四角形をソートする
    tolerance = width / 3 if width < height else width / 6
    rects = sorted(rects, key=cmp_to_key(cmp_frame(tolerance)))
    # コマの部分の画像を切り出す
    frames = []
    for rect in rects:
        frame = cut_frame(image, rect)
        frames.append(frame)
    return frames

In [48]:
def main():
    for root, dirs, files in os.walk(SRC_DIR):
        rel_path = os.path.relpath(root, SRC_DIR)
        dst_dir = os.path.join(DST_DIR, rel_path)
        print(rel_path, dst_dir)
        if not os.path.exists(dst_dir):
            os.mkdir('dst', dst_dir)
        for i, file in enumerate(files):
            if file.startswith('.'):
                continue
            name, ext = os.path.splitext(file)
            page_num = name.split('-')[1]
            page_dir = os.path.join(dst_dir, page_num)
            if not os.path.exists(page_dir):
                os.mkdir(page_dir)
            if ext.lower() not in {'.jpg', '.png'}:
                continue
            file_path = os.path.join(root, file)
            image = cv2.imread(file_path)
            frames = cut_frames(image)
            for i, frame in enumerate(frames):
                dst_path = os.path.join(page_dir, str(i + 1) + ext)
                cv2.imwrite(dst_path, frame)

In [49]:
SRC_DIR = "data/newgame5"
DST_DIR = "output/newgame5_new"
main()

. output/newgame5_new/.


---
# 大コマを切り抜く作業用
が、結局手動でやったほうがはやいということになったので諦めた  
大体が1ページに8コマあるはずなので、8コマないページを特定して見に行って手で切り抜いたほうがいい

In [28]:
large_koma = list(range(9,89, 8)) + [91, 99, 109,117]
large_koma

[9, 17, 25, 33, 41, 49, 57, 65, 73, 81, 91, 99, 109, 117]

In [ ]:
%matplotlib inline

large_koma = list(range(9,89, 8)) + [91, 99, 109,117]
koma = large_koma[1]

image = cv2.imread("data/newgame4/4-{0:04d}.jpg".format(koma))
plt.imshow(image)


thresh = apply_adaptive_threshold(image, radius=5)
cv2.imwrite("output/image-thresh.jpg", thresh)
plt.imshow(thresh)
frames = cut_frames(image)
print(len(frames))
plt.imshow(frames[0])


In [34]:
large_koma

[9, 17, 25, 33, 41, 49, 57, 65, 73, 81, 91, 99, 109, 117]

In [62]:
%matplotlib inline
large_koma = list(range(9,89, 8)) + [91, 99, 109,117]
for koma in large_koma:
    image = cv2.imread("data/newgame4/4-{0:04d}.jpg".format(koma))
    thresh = apply_adaptive_threshold(image)
    cv2.imwrite("output/image-thresh-{}.jpg".format(koma), thresh)

    lines = cv2.HoughLines(image = thresh, rho=0.01, theta=np.pi/180, threshold=100)
    #lines = cv2.HoughLinesP(image = thresh, rho=0.1, theta = np.pi/180, 
    #                                           threshold = 100, minLineLength = 255, maxLineGap = 1)

    image_lines = np.copy(image)
    image_line_contour = np.ones(image.shape[0:2], dtype=np.uint8)
    image_line_contour.fill(255)
    
    for rho,theta in lines[0]:
        a, b = np.cos(theta), np.sin(theta)
        x0, y0 = a*rho, b*rho
        x1, y1 = int(x0 + 1000*(-b)), int(y0 + 1000*(a))
        x2, y2 = int(x0 - 1000*(-b)), int(y0 - 1000*(a))
        #cv2.line(image_lines,(x1,y1),(x2,y2),(0, 255, 0), 1)
        cv2.line(image_line_contour, (x1,y1), (x2,y2), (0, 0,0), 2)

    #for x1,y1,x2,y2 in lines[0]:
    #    #cv2.line(image_lines,(x1, y1), (x2, y2), (0, 255, 0), 3)        
    #    cv2.line(image_line_contour, (x1,y1), (x2,y2), (0, 0,0), 3)
    
    ##cv2.imwrite('output/{}-plines.jpg'.format(koma), image_lines)
    #cv2.imwrite('output/{}-plines.jpg'.format(koma), image_line_contour)
    
    _, contours, hierarchy = cv2.findContours(image_line_contour, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    image_contours = np.copy(image)
    cv2.drawContours(image_contours,contours,-1,(0,255,0),3)
    cv2.imwrite('output/{}-contours.jpg'.format(koma), image_contours)
    #print len(contours),

    #continue
    
    #print len(contours),
    min_perimeter, max_perimeter = (width + height) * 0.25,  (width + height) * 1.5
    rects = extract_rects_from_controus(contours, min_perimeter, max_perimeter)
    f1 = lambda x: 150000 < x[1][0] * x[1][1] < 160000
    f2 = lambda x: 600000 < x[1][0] * x[1][1] < 640000    
    rects = filter(lambda x: f1(x) or f2(x), rects)
    
    # 抽出した四角形をソートする
    tolerance = width / 3 if width < height else width / 6
    rects = sorted(rects, key=cmp_to_key(cmp_frame(tolerance)))
    #print len(rects)
    # コマの部分の画像を切り出す
    frames = []
    for rect in rects:
        frame = cut_frame(image, rect)
        frames.append(frame)

    for i, frame in enumerate(frames):
        cv2.imwrite('temp/{}-{}-frame.jpg'.format(koma,i), frame)

plt.imshow(image_lines)

NameError: name 'width' is not defined